# Lab 1

## Imports

In [34]:
import pandas as pd
from collections import defaultdict
from datetime import datetime
import numpy as np
from heapq import heapify, heappop, heappush
from math import radians, cos, sin, sqrt, atan2
from dataclasses import dataclass


## Data import and analysis
Import of the data from `connection_graph.csv` and dropping unnecessary data (i.e. `company` column). Columns `departure_time` and `arrival_time` for some cells have time that's greater than 24h. To solve this cells with value greater than 24h are normalized so they would have correct value.

In [35]:
def normalize_time(time):
    h, m, s = map(int, time.split(':'))
    h = h % 24
    return datetime.strptime(f"{h:02}:{m:02}:{s:02}", "%H:%M:%S")


df = pd.read_csv('connection_graph.csv', index_col=0, dtype={'line': str})
df = df.drop(columns='company')

df['departure_time'] = df['departure_time'].apply(normalize_time)
df['arrival_time'] = df['arrival_time'].apply(normalize_time)
df = df[df.duplicated(keep=False)]
df

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
0,A,1900-01-01 20:52:00,1900-01-01 20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539
1,A,1900-01-01 20:53:00,1900-01-01 20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735
2,A,1900-01-01 20:54:00,1900-01-01 20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376
3,A,1900-01-01 20:55:00,1900-01-01 20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617
4,A,1900-01-01 20:57:00,1900-01-01 20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383
...,...,...,...,...,...,...,...,...,...
996515,967,1900-01-01 18:38:00,1900-01-01 18:39:00,Smolec - Lipowa/pętla,Smolec - Dębowa (sklep),51.075005,16.884421,51.076932,16.881543
996516,967,1900-01-01 18:39:00,1900-01-01 18:41:00,Smolec - Dębowa (sklep),Krzeptów - skrzy.,51.076932,16.881543,51.088752,16.875808
996517,967,1900-01-01 18:41:00,1900-01-01 18:42:00,Krzeptów - skrzy.,Krzeptów - Boisko,51.088752,16.875808,51.090678,16.885584
996518,967,1900-01-01 18:42:00,1900-01-01 18:43:00,Krzeptów - Boisko,Krzeptów - Dolina Krzeptowa,51.090678,16.885584,51.092268,16.892523


## Exercise 1

### Class `Connection`
Dataclass that holds:
 - `line`: name of the line
 - `departure_time`: time of the departure of starting stop
 - `arrival_time`: time of the arrival in the target stop

In [36]:
@dataclass
class Connection:
    line: str
    departure_time: pd.Timestamp
    arrival_time: pd.Timestamp


### Class `Stop`
It has attributes:
  - `name`: name of the stop
  - `eta` (estimated time of arrival): time of the journey to this stop
  - `position`: latitude and longitude of this stop
  - `stops`: dictionary of names of available stops with possible connections
  - `previous_stop`: previous stop of the path

In [42]:
class Stop:
    def __init__(self, name):
        self.name = name
        self.eta = None
        self.position: tuple[float, float]
        self.stops = defaultdict(list[Connection])
        self.previous_stop = None

    def __repr__(self):
        return f"Stop({self.name}, ETA={self.eta})"

    def reconstruct_path(self):
        path = [self]

        previous_stop = self.previous_stop
        while previous_stop is not None:
            path.append(previous_stop)
            previous_stop = previous_stop.previous_stop

        return path

    def next_stop_eta(self, next_stop, current_time):
        timetable = self.stops[next_stop.name]
        next_stop_eta = None

        for connection in timetable:
            if connection.departure_time >= current_time and next_stop_eta is None:
                tmp_eta = connection.arrival_time - current_time
                if tmp_eta < pd.Timedelta(hours=0):
                    tmp_eta += pd.Timedelta(hours=24)

                next_stop_eta = tmp_eta + self.eta

        if next_stop_eta is None:
            next_stop_eta = timetable[0].arrival_time - current_time + pd.Timedelta(hours=24)

        return next_stop_eta


In [40]:
class Graph:
    def __init__(self, data):
        start_stops = data['start_stop'].unique()
        end_stops = data['end_stop'].unique()
        stops = {stop: Stop(stop) for stop in np.union1d(start_stops, end_stops)}

        for start_stop, group in data.groupby('start_stop'):
            for row in group.itertuples(index=False):
                stops[start_stop].position = (row.start_stop_lat, row.start_stop_lon)
                stops[start_stop].stops[row.end_stop].append(Connection(row.line, row.departure_time, row.arrival_time))

        for node in stops.values():
            for end_stop in node.stops:
                node.stops[end_stop].sort(key=lambda x: x.arrival_time)

        self.stops = stops

    def run_dijkstra(self):
        current_node: Stop = self.stops.pop(start_stop_name)
        current_node.eta = pd.Timedelta(hours=0)
        not_inf_nodes = []
        last_stop_arr = starting_time

        # Updating the neighbour distance
        while current_node.name != final_stop_name:
            for next_stop_name in current_node.stops:
                if next_stop_name in self.stops:
                    next_stop = self.stops[next_stop_name]

                    if next_stop not in not_inf_nodes:
                        not_inf_nodes.append(next_stop)

                    next_stop_eta = current_node.next_stop_eta(next_stop, last_stop_arr)

                    if next_stop.eta is None or next_stop.eta > next_stop_eta:
                        next_stop.eta = next_stop_eta
                        next_stop.previous_stop = current_node

            not_inf_nodes.sort(key=lambda x: x.eta)
            current_node = self.stops.pop(not_inf_nodes.pop(0).name)
            last_stop_arr = starting_time + current_node.eta
        print(starting_time, current_node, last_stop_arr)
        print(current_node.reconstruct_path())

    def a_star(self):
        start_stop: Stop = self.stops[start_stop_name]
        final_stop: Stop = self.stops[final_stop_name]
        start_stop.eta = pd.Timedelta(hours=0)

        open_set = [(0, start_stop)]
        heapify(open_set)

        while open_set:
            current_stop: Stop = heappop(open_set)[1]
            if current_stop.name == final_stop.name:
                return current_stop.reconstruct_path()

            for next_stop_name in current_stop.stops:
                next_stop = self.stops[next_stop_name]
                tentative_g_cost = current_stop.next_stop_eta(next_stop, current_stop.eta + starting_time)
                if next_stop.eta is None or tentative_g_cost < next_stop.eta:
                    next_stop.previous_stop = current_stop
                    next_stop.eta = tentative_g_cost
                    priority = tentative_g_cost.total_seconds() + haversine(next_stop.position, final_stop.position)
                    if next_stop not in [i[1] for i in open_set]:
                        heappush(open_set, (priority, next_stop))
        return None

In [44]:
def haversine(coord1, coord2):
    lat_1, lon_1 = radians(coord1[0]), radians(coord1[1])
    lat_2, lon_2 = radians(coord2[0]), radians(coord2[1])

    a = sin((lat_2 - lat_1) / 2) ** 2 + cos(lat_1) * cos(lat_2) * sin((lon_2 - lon_1) / 2) ** 2
    return 6371 * 2 * atan2(sqrt(a), sqrt(1 - a))


start_stop_name = "Paprotna"
final_stop_name = "Kasprowicza"
starting_time = datetime.strptime("20:53:00", "%H:%M:%S")

y = Graph(df)
y.run_dijkstra()
# print(y.a_star())

1900-01-01 20:53:00 Stop(Kasprowicza, ETA=0 days 00:09:00) 1900-01-01 21:02:00
[Stop(Kasprowicza, ETA=0 days 00:09:00), Stop(Syrokomli, ETA=0 days 00:08:00), Stop(Pola, ETA=0 days 00:07:00), Stop(Broniewskiego, ETA=0 days 00:06:00), Stop(Bałtycka, ETA=0 days 00:04:00), Stop(Bezpieczna, ETA=0 days 00:02:00), Stop(Obornicka (Wołowska), ETA=0 days 00:01:00), Stop(Paprotna, ETA=0 days 00:00:00)]
